[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sierrajulietromeo/N12_plane_spotter/blob/master/N15_Plane_Tracker.ipynb)

# N12 Plane Tracker

I'm using a Raspberry Pi Zero W and a USB TV Tuner to pick up planes flying close to my house. Living in North London we are on the approach for Heathrow and I wanted to collect this data and do things with it.



In [1]:
    
!pip install fiona

    100% |████████████████████████████████| 15.8MB 1.6MB/s 
    100% |████████████████████████████████| 81kB 23.8MB/s 
  Running setup.py bdist_wheel for munch ... - \ done
  Stored in directory: /root/.cache/pip/wheels/db/bf/bc/06a3e1bfe0ab27d2e720ceb3cff3159398d92644c0cec2c125
Successfully built munch


In [0]:
import pandas as pd
import numpy as np

  

In [30]:

df = pd.read_csv('Tab_Delimited_data.csv')

#df = pd.read_csv('AC_data_2019.csv')
df.head()



,Transponder,Messages,Flight,First Speed,Last Speed,Squawk,First Seen,First Latitude,First Longitude,First Altitude,Last Seen,Last Latitude,Last Longitude,Last Altitude,Low Dist,High Dist
0,\t407464,81.0,BAW503,168.0,144.0,7470.0,2018-10-16 20:14:05 Tuesday,"51,465,594","-0,157242",3425.0,2018-10-16 20:17:25 Tuesday,"51,465,454","-0,288317",1000,"8,5","9,4"
1,\t40097f,208.0,BAW7PW,219.0,149.0,5377.0,2018-10-16 20:14:05 Tuesday,"51,384,888","-0,081952",6325.0,2018-10-16 20:20:44 Tuesday,"51,465,268","-0,308609",1175,"8,5","14,2"
2,\t343385,53.0,VLG41TT,401.0,413.0,1071.0,2018-10-16 20:14:05 Tuesday,"50,728,363","-0,340725",21250.0,2018-10-16 20:17:04 Tuesday,"50,642,029","-0,298293",26425,"53,1","58,1"
3,\t44034b,114.0,EZY91PD,456.0,437.0,5212.0,2018-10-16 20:14:05 Tuesday,"50,667,921","-0,493642",35250.0,2018-10-16 20:17:59 Tuesday,"50,339,914","-0,208146",39025,"57,6","76,1"
4,\t47a46e,341.0,NAX53C,458.0,469.0,2376.0,2018-10-16 20:14:05 Tuesday,"50,670,109","-0,725914",37975.0,2018-10-16 20:25:43 Tuesday,"51,659,996","0,054398",38000,"4,2","59,9"


In [13]:
len(df)  # Checking number of records in the dataframe

381

In [35]:
df.Messages = pd.to_numeric(df.Messages, errors='coerce').fillna(0).astype(np.int64)   # convert messages to numeric, fill blanks with a ZERO then convert to integer
df["Last Altitude"] = pd.to_numeric(df["Last Altitude"], errors='coerce').fillna(0).astype(np.int64)   # convert messages to numeric, fill blanks with a ZERO then convert to integer
df["First Altitude"] = pd.to_numeric(df["First Altitude"], errors='coerce').fillna(0).astype(np.int64)   # convert messages to numeric, fill blanks with a ZERO then convert to integer
df["Squawk"] = pd.to_numeric(df["Squawk"], errors='coerce').fillna(0).astype(np.int64)   # convert messages to numeric, fill blanks with a ZERO then convert to integer
df["First Speed"] = pd.to_numeric(df["First Speed"], errors='coerce').fillna(0).astype(np.int64)   # convert messages to numeric, fill blanks with a ZERO then convert to integer
df["Last Speed"] = pd.to_numeric(df["Last Speed"], errors='coerce').fillna(0).astype(np.int64)   # convert messages to numeric, fill blanks with a ZERO then convert to integer

df.head()

,Transponder,Messages,Flight,First Speed,Last Speed,Squawk,First Seen,First Latitude,First Longitude,First Altitude,Last Seen,Last Latitude,Last Longitude,Last Altitude,Low Dist,High Dist
0,\t407464,81,BAW503,168,144,7470,2018-10-16 20:14:05 Tuesday,"51,465,594","-0,157242",3425,2018-10-16 20:17:25 Tuesday,"51,465,454","-0,288317",1000,"8,5","9,4"
1,\t40097f,208,BAW7PW,219,149,5377,2018-10-16 20:14:05 Tuesday,"51,384,888","-0,081952",6325,2018-10-16 20:20:44 Tuesday,"51,465,268","-0,308609",1175,"8,5","14,2"
2,\t343385,53,VLG41TT,401,413,1071,2018-10-16 20:14:05 Tuesday,"50,728,363","-0,340725",21250,2018-10-16 20:17:04 Tuesday,"50,642,029","-0,298293",26425,"53,1","58,1"
3,\t44034b,114,EZY91PD,456,437,5212,2018-10-16 20:14:05 Tuesday,"50,667,921","-0,493642",35250,2018-10-16 20:17:59 Tuesday,"50,339,914","-0,208146",39025,"57,6","76,1"
4,\t47a46e,341,NAX53C,458,469,2376,2018-10-16 20:14:05 Tuesday,"50,670,109","-0,725914",37975,2018-10-16 20:25:43 Tuesday,"51,659,996","0,054398",38000,"4,2","59,9"


In [8]:
print(list(df.columns))

['Transponder', 'Messages', 'Flight', 'First Speed', 'Last Speed', 'Squawk', 'First Seen', 'First Latitude', 'First Longitude', 'First Altitude', 'Last Seen', 'Last Latitude', 'Last Longitude', 'Last Altitude', 'Low Dist', 'High Dist']


In [12]:
# Create new dataframe based on DF of all the planes that I first detect between 5000 and 10000 ft high - 5000 might be a bit low
df2 = df[df['First Altitude'].between(5000, 10000, inclusive=True)]
#df2.head()
len(df2)

,Transponder,Messages,Flight,First Speed,Last Speed,Squawk,First Seen,First Latitude,First Longitude,First Altitude,Last Seen,Last Latitude,Last Longitude,Last Altitude,Low Dist,High Dist
1,\t40097f,208.0,BAW7PW,219.0,149.0,5377.0,2018-10-16 20:14:05 Tuesday,"51,384,888","-0,081952",6325.0,2018-10-16 20:20:44 Tuesday,"51,465,268","-0,308609",1175,"8,5","14,2"
6,\t4010db,418.0,BAW607,254.0,129.0,2741.0,2018-10-16 20:14:07 Tuesday,"51,353,531","0,219578",10000.0,2018-10-16 20:27:10 Tuesday,"51,465,271","-0,309473",925,"8,5","23,6"
8,\t407534,113.0,BAW870,288.0,454.0,1070.0,2018-10-16 20:14:07 Tuesday,"51,185,861","-0,004196",9600.0,2018-10-16 20:20:27 Tuesday,"51,183,068","0,892487",25675,"26,1","47,5"
10,\t405a49,311.0,BAW349,227.0,146.0,6752.0,2018-10-16 20:14:08 Tuesday,"51,337,156","-0,009613",7975.0,2018-10-16 20:23:28 Tuesday,"51,465,225","-0,34451",925,"8,5","17,4"
11,\t406a3e,257.0,BAW19LX,238.0,149.0,3507.0,2018-10-16 20:14:08 Tuesday,"51,615,417","-0,19471",6975.0,2018-10-16 20:22:20 Tuesday,"51,465,271","-0,313185",925,"0,4","9,9"


In [10]:
# Then refine that dataframe further to planes where last altitude detected was between 0-1000ft.   Now the dataset is of planes that overflew my house
df2 = df2[df2['Last Altitude'].between(0, 1000, inclusive=True)]
#df2.head(20)
len(df2)   # length of dataframe

TypeError: ignored

In [11]:
import fiona
c = fiona.open('ne_10m_airports.shp', 'r')

OSError: ignored